# Regression model for SOH prediction 

For a resource-constrained device like the ESP32, random forest regression is generally a better choice compared to RNN, because it is computationally lighter and easier to implement. 

 we will use the Random Forest regression model to predict the state of health (SOH) of a battery. 


In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


about the data:

In [31]:
df = pd.read_csv('data.csv')
display(df.head())
display(df.describe())
display(df.info())


,Voltage,Current,Battery Temperature (°C),Environmental Temperature (°C),Cycle Count,Battery Age (Days),Unnamed: 6,SOC (%),SOH (%)
0,3.2,0.50,25,22,10,30,NaN,20,98
1,3.3,0.60,26,23,20,60,NaN,30,97
2,3.4,0.58,26,23,25,75,NaN,35,96
3,3.5,0.55,27,23,30,90,NaN,40,96
4,3.6,0.60,27,24,35,105,NaN,45,95


,Voltage,Current,Battery Temperature (°C),Environmental Temperature (°C),Cycle Count,Battery Age (Days),Unnamed: 6,SOC (%),SOH (%)
count,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,0.0,45.000000,45.000000
mean,3.960667,0.763778,30.444444,25.755556,92.488889,278.000000,NaN,67.822222,88.244444
std,0.297798,0.186344,3.539103,2.970733,42.976538,129.484994,NaN,22.708918,5.697510
min,3.200000,0.400000,25.000000,20.000000,10.000000,30.000000,NaN,20.000000,77.000000
25%,3.750000,0.620000,28.000000,24.000000,55.000000,165.000000,NaN,50.000000,84.000000
50%,4.000000,0.760000,30.000000,26.000000,95.000000,285.000000,NaN,70.000000,89.000000
75%,4.200000,0.900000,33.000000,28.000000,130.000000,390.000000,NaN,90.000000,93.000000
max,4.400000,1.100000,38.000000,32.000000,158.000000,480.000000,NaN,99.000000,98.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Voltage                         45 non-null     float64
 1   Current                         45 non-null     float64
 2   Battery Temperature (°C)        45 non-null     int64  
 3   Environmental Temperature (°C)  45 non-null     int64  
 4   Cycle Count                     45 non-null     int64  
 5   Battery Age (Days)              45 non-null     int64  
 6   Unnamed: 6                      0 non-null      float64
 7   SOC (%)                         45 non-null     int64  
 8   SOH (%)                         45 non-null     int64  
dtypes: float64(3), int64(6)
memory usage: 3.3 KB


None

lets remove unnecessary columns and check the correlation between the columns.

In [32]:
df.drop(['Unnamed: 6'], axis=1, inplace=True)
df.dropna(inplace=True)
display(df.info())
display(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Voltage                         45 non-null     float64
 1   Current                         45 non-null     float64
 2   Battery Temperature (°C)        45 non-null     int64  
 3   Environmental Temperature (°C)  45 non-null     int64  
 4   Cycle Count                     45 non-null     int64  
 5   Battery Age (Days)              45 non-null     int64  
 6   SOC (%)                         45 non-null     int64  
 7   SOH (%)                         45 non-null     int64  
dtypes: float64(2), int64(6)
memory usage: 2.9 KB


None

Index(['Voltage', 'Current', 'Battery Temperature (°C)',
       'Environmental Temperature (°C)', 'Cycle Count', 'Battery Age (Days)',
       'SOC (%)', 'SOH (%)'],
      dtype='object')

Data Splitting:

In [33]:
X = df.drop(['SOH (%)'], axis=1)
y = df['SOH (%)']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


standardisation:

In [34]:
SC = StandardScaler()
x_train_scaled= SC.fit_transform(x_train)
x_test_scaled = SC.transform(x_test)


Fitting the dataset and training the model:

In [35]:
regression = RandomForestRegressor(n_estimators=100, random_state=42)
model = regression.fit(x_train_scaled, y_train)
y_pred = model.predict(x_test_scaled)   

Model evaluation:
for regression models, evaluation metrics are different from classification models. We will use the following metrics to evaluate the model:
- Mean Squared Error
- R2 – Score

In [36]:
mean_squared_error(y_test, y_pred)
r2_score(y_test, y_pred)
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('R2 Score:', r2_score(y_test, y_pred))


Mean Squared Error: 0.26238888888888806
R2 Score: 0.9928051794177387


Lets not import this model using joblib

In [37]:
import joblib
joblib.dump(model, 'AURAmodel.pkl')

['AURAmodel.pkl']